<a href="https://colab.research.google.com/github/ykitaguchi77/FundusPhoto/blob/main/SeqNet1.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Running Github codes (SeqNet) in Google Colab



In [1]:
#google driveをcolabolatoryにマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Githubよりリポジトリをcloneする
!git clone https://github.com/conscienceli/SeqNet.git

Cloning into 'SeqNet'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 48 (delta 15), reused 41 (delta 10), pack-reused 0
Unpacking objects: 100% (48/48), done.


#作業ディレクトリを移動

In [3]:
import os
import shutil

model_path = '/content/SeqNet'

#作業ディレクトリをpathに移動する
os.chdir(model_path)

#作業ディレクトリ直下のファイルを確認
!ls

LICENSE  predict.py  requirements-conda.txt  train.py
pics	 README.md   requirements-pip.txt    utils


In [4]:
#評価画像用のフォルダを作成
data_dir = "./data/test_images/"

if os.path.exists(data_dir):
    shutil.rmtree(data_dir)
os.makedirs(data_dir)

#copy example data
#shutil.copy("/content/drive/MyDrive/Deep_learning/FundusPhoto/cropped_img/img00085008_00_1R.jpg", "./data/test_images/image1.jpg")

In [ ]:
#入力用フォルダを定義
orig_path = "/content/drive/MyDrive/Deep_learning/FundusPhoto/cropped_img_2/"
total_num = len(os.listdir(orig_path))
print("total images: "+str(total_num))
img_num = list(range(0, total_num, 1)) #処理する画像の番号を指定(0, 1414, 1)
#img_num = list(range(0, 2, 1)) #少量の画像でテスト

img_list = []
for i in img_num:
    img_list.append(os.listdir(orig_path)[i])
print(img_list)

k=0
for i in img_list:
    k+=1
    shutil.copy(orig_path +"/" + str(i), data_dir + str(i))
    print(str(k)+" images copied!")


#出力用フォルダを作る
dst_path = "/content/drive/MyDrive/Deep_learning/FundusPhoto/vascular_img_2"
if os.path.exists(dst_path):
    shutil.rmtree(dst_path)
os.makedirs(dst_path, exist_ok=True)

In [6]:
#Download pretrained model from Gdrive
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

file_id = '1OYjzu0gixtga6e7Rvb2mZoSSYJkXWRNB'

os.makedirs('./trained_model/ALL', exist_ok=True)
destination = './trained_model/ALL/Final_Emer_Iteration_3_cropsize_128_epochs_200.hdf5'
download_file_from_google_drive(file_id, destination)


#upgrade modules
!pip3 install --upgrade tensorflow==1.15.0
!pip3 install --upgrade keras==2.3.1
!pip3 install --upgrade h5py==2.10.0

#stride_sizeを増やす事によるタイムコスト削減
#!cat -n predict.py # -n はnumberの意味
#!sed -i -e "133c predict(batch_size=24, epochs=200, iteration=3, stride_size=10, crop_size=128," predict.py

#書き換えたpredict.pyで置換
shutil.copy("/content/drive/MyDrive/Deep_learning/FundusPhoto/SeqNet/predict_png.py", "/content/SeqNet/predict.py")

     |████████████████████████████████| 412.3 MB 23 kB/s 
     |████████████████████████████████| 3.8 MB 52.1 MB/s 
     |████████████████████████████████| 50 kB 8.8 MB/s 
     |████████████████████████████████| 503 kB 66.1 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=1981b7a02ff1ee24ac4a238967d82910b77abe6dbc55170eed66aaf9ba53c6ce
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

'/content/SeqNet/predict.py'

In [ ]:
"""
#ここは不要
!cat -n requirements-pip.txt # -n はnumberの意味

!sed -i -e "43c protobuf==3.11.3" requirements-pip.txt # protobufはバージョン違いしかないので変更
!sed -i -e "47d" requirements-pip.txt # pyGPUはサポートされていないので除外

!cat -n requirements-pip.txt # -n はnumberの意味

!pip3 install -r requirements-pip.txt
"""


#**Run codes**

In [7]:
"""
これを走らせる前に、predict.pyを書き換える。
・artery, vein finalを削除
・出力ファイル形式を.jpgに変更
・stride_sizeを0に変更
"""
!python predict.py -i /content/SeqNet/data/test_images -o /content/drive/MyDrive/Deep_learning/FundusPhoto/vascular_img_2
#!python predict.py -i data_dir -o /content/drive/MyDrive/Deep_learning/FundusPhoto/vascular_img_2/

Using TensorFlow backend.



2021-10-02 03:43:23.740545: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-10-02 03:43:23.744856: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-10-02 03:43:23.745132: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5637e92a4a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-10-02 03:43:23.745165: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-10-02 03:43:23.746855: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-10-02 03:43:24.046335: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero